In [2]:
import os

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
model = ChatGoogleGenerativeAI(model='gemini-pro', convert_system_message_to_human= True)

In [7]:
response = model.invoke("Hello, world!").content

In [8]:
from langchain_core.output_parsers import StrOutputParser

In [9]:
parser = StrOutputParser()

In [10]:
chain = model | parser 

In [11]:
chain.invoke("Hello, world!")

''

In [12]:
from langchain_core.messages import HumanMessage

In [16]:
while True:
    message = input("Write your query")
    if message == "Bye":
        break
    else:
        print(chain.invoke([HumanMessage(content=message)]))

Hello there! How can I assist you today?
Yo, what's good?


In [17]:
from langchain_core.chat_history import BaseChatMessageHistory

In [18]:
from langchain_core.chat_history import InMemoryChatMessageHistory

In [19]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [20]:
from langchain_core.messages import AIMessage

In [21]:
output = model.invoke([
        HumanMessage(content="Hi! I'm cristinao"),
        AIMessage(content="Hello cristinao! How can I assist you today?"),
        HumanMessage(content="What's my name?") 
])

In [22]:
parser.invoke(output)

'Your name is cristinao. Is there anything else I can help you with today?'

In [23]:
store ={}

In [24]:
def get_session_history(session_id: str) ->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [25]:
model_with_memory = RunnableWithMessageHistory(model, get_session_history)

In [26]:
config = {"configurable": {"session_id": "firstchat"}}

In [27]:
model_with_memory.invoke([HumanMessage(content="Hi! I am Samin Sadaf")], config=config).content

'Hello Samin Sadaf, it is nice to meet you!'

In [29]:
model_with_memory.invoke([HumanMessage(content="What is my name?")], config=config).content

'Your name is Samin Sadaf.'

In [30]:
config = {"configurable": {"session_id": "secondchat"}}

In [31]:
model_with_memory.invoke([HumanMessage(content="What is my name?")], config=config).content

'I do not have access to your personal information, so I cannot answer this question.'

In [32]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi! I am Samin Sadaf'), AIMessage(content='Hello Samin Sadaf, it is nice to meet you!', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-32a900da-cf48-417a-b462-9037f7db3ca7-0', usage_metadata={'input_tokens': 9, 'output_tokens': 13, 'total_tokens': 22}), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Samin Sadaf.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [

In [33]:
config = {"configurable": {"session_id": "firstchat"}}

In [35]:
model_with_memory.invoke([HumanMessage(content="What did I ask you last time?What was your answer?")], config=config).content

'**Question:** What is my name?\n\n**Answer:** Your name is Samin Sadaf.'

In [36]:
config = {"configurable": {"session_id": "secondchat"}}

In [37]:
model_with_memory.invoke([HumanMessage(content="What did I ask you last time?What was your answer?")], config=config).content

'I do not have a memory of previous interactions with you, so I cannot answer this question.'

In [38]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi! I am Samin Sadaf'), AIMessage(content='Hello Samin Sadaf, it is nice to meet you!', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-32a900da-cf48-417a-b462-9037f7db3ca7-0', usage_metadata={'input_tokens': 9, 'output_tokens': 13, 'total_tokens': 22}), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Samin Sadaf.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [

In [39]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [40]:
prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)